<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/causal-inference/dowhy_conditional_treatment_effects_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conditional Average Treatment Effect (CATE)

This notebook leverages EconML's CATE estimation using different methods.

We will follow the typical doWhy workflow: model, identify, estimate, and refute.

## Setup Dependencies

In [2]:
!sudo apt install graphviz libgraphviz-dev graphviz-dev pkg-config -y -q

!pip install -q pygraphviz --install-option="--include-path=/usr/include/graphviz" \
--install-option="--library-path=/usr/lib/graphviz/"
!pip install -q dowhy

Reading package lists...
Building dependency tree...
Reading state information...
pkg-config is already the newest version (0.29.1-0ubuntu2).
graphviz is already the newest version (2.40.1-2).
libgraphviz-dev is already the newest version (2.40.1-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
     |████████████████████████████████| 71kB 3.0MB/s 
     |████████████████████████████████| 286kB 15.1MB/s 
     |████████████████████████████████| 5.6MB 21.6MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
ERROR: econml 0.6.1 has requirement matplotlib<3.1, but you'll have matplotlib 3.1.

In [0]:
import 

